In [1]:
#!pip install undetected-chromedriver

import pandas as pd
import csv
import json
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import re # regex
from datetime import datetime 
from openpyxl import load_workbook

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, relationship, Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Boolean, Integer, Float, String, DateTime, ForeignKey
from sqlalchemy import MetaData
from sqlalchemy.exc import OperationalError

In [2]:

# creating classes corresponding to the actual tables of the SSMS database
# Each class has attributes representing the columns in the respective tables
# and relationships are established using the relationship function.

Base = declarative_base()

class Retailer(Base):
    __tablename__ = 'Retailers'
    __tableargs__ = {"extend_existing": True}
    RetailerID = Column(Integer, primary_key=True)
    RetailerName = Column(String, unique=True)
    Links = relationship('Links', back_populates='Retailer')

class Product(Base):
    __tablename__ = 'Products'
    __tableargs__ = {"extend_existing": True}
    ProductID = Column(Integer, primary_key=True)
    Product = Column(String(collation='SQL_Latin1_General_CP1_CI_AS'))
    Brand = Column(String)
    Volume = Column(Float)
    Measure = Column(String)
    Units = Column(Integer)    

class Links(Base):
    __tablename__ = 'Links'
    __tableargs__ = {"extend_existing": True}
    LinkID = Column(Integer, primary_key=True, autoincrement=True)
    Link = Column(String)  
    ProductID = Column(Integer, ForeignKey('Products.ProductID'))
    RetailerID = Column(Integer, ForeignKey('Retailers.RetailerID'))
    Retailer = relationship('Retailer', back_populates='Links')
    
class PriceArchive(Base):
    __tablename__ = "PriceArchive"
    __tableargs__ = {"extend_existing": True}
    id = Column(Integer, primary_key=True, autoincrement=True)
    ProductID = Column(Integer, ForeignKey('Products.ProductID'))
    RetailerID = Column(Integer, ForeignKey('Retailers.RetailerID'))
    FinalPrice = Column(Float)
    InitialPrice = Column(Float)
    Datestamp = Column(DateTime)
    
# Define your database connection string
conn_str = 'DRIVER={ODBC DRIVER 17 for SQL Server};SERVER=KITSAKIS;DATABASE=price_tracking;UID=kitsakis;PWD=priceTracker' 
connection_url = f'mssql+pyodbc:///?odbc_connect={conn_str}'

# Create SQLAlchemy engine
pyodbc_engine = create_engine(connection_url, echo=True)

Base.metadata.create_all(bind=pyodbc_engine)
Session = sessionmaker(bind=pyodbc_engine)
session = Session()

# Test the connection by querying the database
try:
    with pyodbc_engine.connect() as connection:
        print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")


In [3]:

# try:
#     # Attempt to create the engine
#     pyodbc_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}', echo=False)
    
#     # Test the connection by creating a session
#     Session = sessionmaker(bind=pyodbc_engine)
#     session = Session()
    
#     # Close the session
#     session.close()
    
#     print("Database connection successful!")
# except OperationalError as e:
#     print(f"Database connection failed: {e}")


In [4]:

from selenium import webdriver

# Set the desired timeouts (in seconds)
#timeout = 60

# Create an Internet Explorer driver instance
driver = webdriver.Edge()

#driver.implicitly_wait(timeout)  # Set implicit wait timeout


# Now you can use 'driver' for interacting with Internet Explorer

In [5]:
# Get today's date
today_date = datetime.now().strftime('%Y-%m-%d')

In [ ]:
try:
    for link_instance in session.query(Links).all():
        try:
            driver.get(link_instance.Link)

            product = {
                    "productID" : "",
                    "retailerID" : "",
                    "final_price" : "",
                    "initial_price" : ""
                }      

            if link_instance.RetailerID == 1: #Sklavenitis

                time.sleep(4) # load the page 

                try:
                    # fetch final price
                    final_price_element = driver.find_element(By.CSS_SELECTOR, "div.product-detail-outer  div.main-price > div.price")

                    # replace comma with full stop 
                    product["final_price"] = float(final_price_element.get_attribute('data-price').replace(",", "."))
                except:
                    product["final_price"] = None 

                # fetch initial price
                try:
                    product["initial_price"] = float((driver.find_element(By.CSS_SELECTOR, "div.product-detail-outer  div.main-price > div.deleted > div.deleted__price").text).replace(",", "."))
                except:
                    product["initial_price"] = None

                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       

            if link_instance.RetailerID == 2: #AB

                time.sleep(6) # load the page 

                # fetch final price
                try:
                    final_price_element = driver.find_elements(By.CSS_SELECTOR, "[data-testid = 'product-properties-price-info'] > [data-testid = 'product-block-price']")[0].text
                    # replace comma with full stop 
                    product["final_price"] = float(final_price_element.strip(" €").replace(",", "."))
                except:
                    product["final_price"] = None 

                # fetch initial price
                try:
                    initial_price_element = driver.find_element(By.CSS_SELECTOR, ".sc-e3oax-20 > [data-testid = 'product-block-old-price']").text
                    product["initial_price"] = float(initial_price_element.strip(" €").replace(",", "."))
                except:
                    product["initial_price"] = None


                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       

            if link_instance.RetailerID == 3: #MyMarket

                time.sleep(4) # load the page 

                # Try to close the cookies info popup window
                try:
                    driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll").click()
                except:
                    pass

                # fetch final price 
                try:
                    product["final_price"] = float((driver.find_element(By.CSS_SELECTOR, "span.product-full--final-price").text).strip("€").replace(",", "."))
                except:
                    product["final_price"] = None

                # fetch initial price
                try:
                    # Locate the div containing a span with exactly the text "Αρχική τιμή"
                    parent_div = driver.find_element(By.XPATH, "//div[span[text()='Αρχική τιμή']]")

                    # Locate the span within this div with the specified class
                    price_span = parent_div.find_element(By.CSS_SELECTOR, 'span.font-bold.diagonal-line.text-base')

                    # Get the text of this span
                    product["initial_price"]  = float(price_span.text.strip("€").replace(",", "."))

                    #product["initial_price"] = float((driver.find_element(By.CSS_SELECTOR, "div.flex-gap-2-justify-end > div.rounded").text).strip("€").replace(",", "."))
                except:
                    product["initial_price"] = None
                    
                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       

            if link_instance.RetailerID == 4: #Masoutis

                time.sleep(4) # load the page 

                # fetch final price
                try:
                    elements = driver.find_element(By.CSS_SELECTOR, "div.item-DscntPrice")
                    if elements.text != '':
                        product["final_price"] = float((elements.text).strip("€ "))
                    else:
                        product["final_price"] = float((driver.find_element(By.CSS_SELECTOR, "div.item-priceContainer > div.item-price").text).strip("€ "))
                except:
                    product["final_price"] = float((driver.find_element(By.CSS_SELECTOR, "div.item-priceContainer > div.item-price").text).strip("€ "))

                # fetch initial price
                try:
                    elements = driver.find_element(By.CSS_SELECTOR, "div.item-StartPrice")
                    if elements.text != '':
                        product["initial_price"] = float((elements.text).strip("€ "))
                    else:
                        product_data["initial_price"] = None
                except:
                    product["initial_price"] = None

                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       


            if link_instance.RetailerID == 5: #Kritikos

                time.sleep(9) # load the page 

                error_message = driver.find_elements(By.CSS_SELECTOR, ".ErrorPageMain_errorText__m9W4T")
                if len(error_message)>0:
                    product["final_price"] = None
                    product["initial_price"] = None

                else:
                    # fetch final price
                    product["final_price"] = float((driver.find_element(By.CSS_SELECTOR, "span.ProductDetails_price__9wMeq").text).strip("€ "))

                    # fetch initial price
                    try:
                        product["initial_price"] = float(re.search(r'(\d+\.\d+)', driver.find_element(By.CSS_SELECTOR, "span.ProductDetails_lineThroughText__fr44S").text.strip("€ "))[0])
                    except:
                        product["initial_price"] = None

                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       


            if link_instance.RetailerID == 6: #MarketIn

                time.sleep(4) # load the page 

                # fetch final price
                try:
                    product["final_price"] = float((driver.find_element(By.CSS_SELECTOR, ".p-price").text).strip(" €").replace(",", "."))
                except:
                    product["final_price"] = None

                # fetch initial price
                try:
                    product["initial_price"] = float((driver.find_element(By.CSS_SELECTOR, ".p-old-price").text).strip(" €").replace(",", "."))
                except:
                    product["initial_price"] = None

                product["productID"] = link_instance.ProductID
                product["retailerID"] = link_instance.RetailerID

                 #for testing the reliability
                print(product["final_price"], product["initial_price"], product["productID"], product["retailerID"])       

            price_archive_instance = PriceArchive(
                ProductID = product["productID"],
                RetailerID = product["retailerID"],
                FinalPrice = product["final_price"],
                InitialPrice = product["initial_price"],
                Datestamp = today_date
            )
            session.add(price_archive_instance)
            session.commit()

        except Exception as e:
            print(f"Error processing link {link_instance.Link}: {str(e)}")
            # Handle the error as needed, e.g., log the error

        
finally:
    # Close the browser to release resources
    driver.quit()
    session.rollback()
    session.close() 

In [ ]:
session.rollback()


In [ ]:
driver.quit()